In [0]:
import numpy as np
import pandas as pd

In [0]:
# defining simple univariate LSTM model
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)


In [0]:
# define a simple input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose a number of time steps
n_steps = 3

# split into samples
X, y = split_sequence(raw_seq, n_steps)

# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


In [0]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [0]:
from tensorflow import keras
from tensorflow.python.keras.layers import Input, Dense, Dropout, LSTM
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.python.keras import optimizers

In [0]:
# define single layer LSTM model
# the hidden layer should have the input like [samples, timesteps, features]

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features))) # 50 units
model.add(Dense(1)) # means only output one value
model.compile(optimizer='adam', loss='mse')

In [0]:
# fit model
model.fit(X, y, epochs=300, verbose=2)

Train on 6 samples
Epoch 1/300
6/6 - 3s - loss: 4596.8613
Epoch 2/300
6/6 - 0s - loss: 4560.2124
Epoch 3/300
6/6 - 0s - loss: 4523.5142
Epoch 4/300
6/6 - 0s - loss: 4486.0649
Epoch 5/300
6/6 - 0s - loss: 4448.0679
Epoch 6/300
6/6 - 0s - loss: 4409.4214
Epoch 7/300
6/6 - 0s - loss: 4370.7778
Epoch 8/300
6/6 - 0s - loss: 4331.8716
Epoch 9/300
6/6 - 0s - loss: 4292.5186
Epoch 10/300
6/6 - 0s - loss: 4252.9165
Epoch 11/300
6/6 - 0s - loss: 4211.4116
Epoch 12/300
6/6 - 0s - loss: 4168.1987
Epoch 13/300
6/6 - 0s - loss: 4124.2632
Epoch 14/300
6/6 - 0s - loss: 4078.7717
Epoch 15/300
6/6 - 0s - loss: 4031.2334
Epoch 16/300
6/6 - 0s - loss: 3981.5203
Epoch 17/300
6/6 - 0s - loss: 3929.6836
Epoch 18/300
6/6 - 0s - loss: 3875.0696
Epoch 19/300
6/6 - 0s - loss: 3820.8406
Epoch 20/300
6/6 - 0s - loss: 3765.2507
Epoch 21/300
6/6 - 0s - loss: 3707.5461
Epoch 22/300
6/6 - 0s - loss: 3648.4258
Epoch 23/300
6/6 - 0s - loss: 3585.7566
Epoch 24/300
6/6 - 0s - loss: 3519.9929
Epoch 25/300
6/6 - 0s - loss: 

In [0]:
# prediction
x_input = array([70, 80, 90])

# the model accpets input as [samples, timesteps, features]
x_input = x_input.reshape((1, n_steps, n_features))

yhat = model.predict(x_input, verbose=2)
print(yhat)

1/1 - 1s
[[102.622086]]


In [0]:
## from above we can see that the model predicts very close to the value we expected
## given that the dataset is just a toy example
## a single layer LSTM is good enough

In [0]:
# define univariate stacked LSTM model

model2 = Sequential()
model2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
# return_seq=True to add more value in the output that can use as input to next layer
model2.add(LSTM(50, activation='relu'))
model2.add(Dense(1)) # still output single value
model2.compile(optimizer='adam', loss='mse')

In [0]:
# fit model
model2.fit(X, y, epochs=150, verbose=2)

In [0]:
# prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model2.predict(x_input, verbose=2)

print(yhat)

1/1 - 1s
[[102.17309]]


In [0]:
## we then compare the different perofrmances of these two model
## given that the dataset is just a toy example
## a single layer LSTM is good enough

In [0]:
# define univariate bidirectional LSTM model
from tensorflow.python.keras.layers import Bidirectional

model3 = Sequential()
# wrap the first hidden layer in bidirecitonal layer
model3.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(n_steps, n_features)))
model3.add(Dense(1))
model3.compile(optimizer='adam', loss='mse')

In [0]:
# fit model
model3.fit(X, y, epochs=200, verbose=2)

In [0]:
# prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=2)

print(yhat)

1/1 - 0s
[[102.622086]]


In [0]:
# define multivariate LSTM model
# dataset generation

in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
from numpy import hstack
dataset = hstack((in_seq1, in_seq2, out_seq))

print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [0]:
# split a multivariate sequence into samples

def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break

        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)

In [0]:
# choose a number of time steps
n_steps = 3

In [0]:
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [0]:
n_features = X.shape[2]

In [0]:
# define LSTM model for multivariate dataset
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [0]:
# fit model
model.fit(X, y, epochs=200, verbose=2)

Train on 7 samples
Epoch 1/200
7/7 - 4s - loss: 17388.7500
Epoch 2/200
7/7 - 0s - loss: 17062.9023
Epoch 3/200
7/7 - 0s - loss: 16775.4844
Epoch 4/200
7/7 - 0s - loss: 16490.3477
Epoch 5/200
7/7 - 0s - loss: 16186.4434
Epoch 6/200
7/7 - 0s - loss: 15831.6230
Epoch 7/200
7/7 - 0s - loss: 15414.6533
Epoch 8/200
7/7 - 0s - loss: 14921.7676
Epoch 9/200
7/7 - 0s - loss: 14368.2900
Epoch 10/200
7/7 - 0s - loss: 13796.4043
Epoch 11/200
7/7 - 0s - loss: 13161.5830
Epoch 12/200
7/7 - 0s - loss: 12438.1465
Epoch 13/200
7/7 - 0s - loss: 11570.6787
Epoch 14/200
7/7 - 0s - loss: 10510.8115
Epoch 15/200
7/7 - 0s - loss: 9284.1514
Epoch 16/200
7/7 - 0s - loss: 7941.7021
Epoch 17/200
7/7 - 0s - loss: 6481.1284
Epoch 18/200
7/7 - 0s - loss: 4902.2905
Epoch 19/200
7/7 - 0s - loss: 3410.5391
Epoch 20/200
7/7 - 0s - loss: 1991.7932
Epoch 21/200
7/7 - 0s - loss: 780.7370
Epoch 22/200
7/7 - 0s - loss: 191.6523
Epoch 23/200
7/7 - 0s - loss: 479.5687
Epoch 24/200
7/7 - 0s - loss: 1159.6084
Epoch 25/200
7/7 - 

In [0]:
# prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=2)
print(yhat)

1/1 - 1s
[[207.91911]]


In [0]:
## LSTM can also be used to predict multiple timesteps in the future
## one choice is to use multi-step vector-output LSTM model
## or Encoder-Decoder LSTM model

In [0]:
# a univariate multi-step vector-output stacked lstm example
# first, again split a univariate sequence into samples

def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out

        if out_end_ix > len(sequence):
            break

        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
        
    return array(X), array(y)

In [0]:
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]

# choose the number of time steps for input and output
n_steps_in, n_steps_out = 3, 2

# split into samples and reshape to the form of model input
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [0]:
# model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [0]:
model.fit(X, y, epochs=100, verbose=2)

Train on 5 samples
Epoch 1/100
5/5 - 4s - loss: 4404.3711
Epoch 2/100
5/5 - 0s - loss: 4364.3667
Epoch 3/100
5/5 - 0s - loss: 4323.2446
Epoch 4/100
5/5 - 0s - loss: 4280.0566
Epoch 5/100
5/5 - 0s - loss: 4233.9272
Epoch 6/100
5/5 - 0s - loss: 4183.9414
Epoch 7/100
5/5 - 0s - loss: 4128.7734
Epoch 8/100
5/5 - 0s - loss: 4066.2292
Epoch 9/100
5/5 - 0s - loss: 3993.3511
Epoch 10/100
5/5 - 0s - loss: 3907.4727
Epoch 11/100
5/5 - 0s - loss: 3806.3687
Epoch 12/100
5/5 - 0s - loss: 3684.9609
Epoch 13/100
5/5 - 0s - loss: 3540.2629
Epoch 14/100
5/5 - 0s - loss: 3368.2324
Epoch 15/100
5/5 - 0s - loss: 3163.5454
Epoch 16/100
5/5 - 0s - loss: 2921.1023
Epoch 17/100
5/5 - 0s - loss: 2648.8379
Epoch 18/100
5/5 - 0s - loss: 2352.1421
Epoch 19/100
5/5 - 0s - loss: 2034.2800
Epoch 20/100
5/5 - 0s - loss: 1696.3828
Epoch 21/100
5/5 - 0s - loss: 1352.1844
Epoch 22/100
5/5 - 0s - loss: 1011.1064
Epoch 23/100
5/5 - 0s - loss: 700.9982
Epoch 24/100
5/5 - 0s - loss: 443.9359
Epoch 25/100
5/5 - 0s - loss: 27

In [0]:
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=2)
print(yhat)

1/1 - 1s
[[112.23638 125.94245]]


In [0]:
# next the Encoder-Decoder LSTM model mostly deals with seq2seq prediction problem
# needs RepeatVector and TimeDistributed modules to solve if input sequences and output sequences have different lengths

In [0]:
# univariate multi-step encoder-decoder lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [0]:
# prepare the dataset

def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequence):
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)

    return array(X), array(y)
 
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
n_steps_in, n_steps_out = 3, 2

X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

In [0]:
# model

model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out)) #
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')


In [0]:
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
 - 5s - loss: 4442.2295
Epoch 2/100
 - 0s - loss: 4416.1948
Epoch 3/100
 - 0s - loss: 4385.6416
Epoch 4/100
 - 0s - loss: 4353.1714
Epoch 5/100
 - 0s - loss: 4318.2632
Epoch 6/100
 - 0s - loss: 4281.2539
Epoch 7/100
 - 0s - loss: 4240.9360
Epoch 8/100
 - 0s - loss: 4195.8311
Epoch 9/100
 - 0s - loss: 4145.0088
Epoch 10/100
 - 0s - loss: 4087.7573
Epoch 11/100
 - 0s - loss: 4020.7168
Epoch 12/100
 - 0s - loss: 3942.5259
Epoch 13/100
 - 0s - loss: 3849.7769
Epoch 14/100
 - 0s - loss: 3739.9780
Epoch 15/100
 - 0s - loss: 3609.9292
Epoch 16/100
 - 0s - loss: 3456.8008
Epoch 17/100
 - 0s - loss: 3276.3635
Epoch 18/100
 - 0s - loss: 3065.3640
Epoch 19/100
 - 0s - loss: 2827.7183
Epoch 20/100
 - 0s - loss: 2562.4644
Epoch 21/100
 - 0s - loss: 2272.4658
Epoch 22/100
 - 0s - loss: 1967.7123
Epoch 23/100
 - 0s - loss: 1656.9746
Epoch 24/100
 - 0s - loss: 1350.7693
Epoch 25/100
 - 0s - loss: 1063.0745
Epoch 26/100
 - 0s - loss: 824.0496
Epoch 27/100
 - 0s - loss: 684.4745
Epoch 28/100

In [0]:
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=2)
print(yhat)

[[[103.17423]
  [125.56825]]]


In [0]:
# multivariate multi-step data preparation
from numpy import array
from numpy import hstack
 
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        if out_end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
        
    return array(X), array(y)

In [0]:
# generate input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

n_steps_in, n_steps_out = 3, 2
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
#print(X.shape, y.shape)

In [0]:
#for i in range(len(X)):
#    print(X[i], y[i])

In [0]:
n_features = X.shape[2]

In [0]:
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [0]:
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
 - 5s - loss: 16794.9551
Epoch 2/200
 - 0s - loss: 16585.8516
Epoch 3/200
 - 0s - loss: 16397.5527
Epoch 4/200
 - 0s - loss: 16205.6250
Epoch 5/200
 - 0s - loss: 15988.0234
Epoch 6/200
 - 0s - loss: 15738.9609
Epoch 7/200
 - 0s - loss: 15450.4697
Epoch 8/200
 - 0s - loss: 15110.4736
Epoch 9/200
 - 0s - loss: 14705.7783
Epoch 10/200
 - 0s - loss: 14236.9424
Epoch 11/200
 - 0s - loss: 13679.0947
Epoch 12/200
 - 0s - loss: 12996.7500
Epoch 13/200
 - 0s - loss: 12183.6787
Epoch 14/200
 - 0s - loss: 11213.1260
Epoch 15/200
 - 0s - loss: 10115.4580
Epoch 16/200
 - 0s - loss: 8845.4209
Epoch 17/200
 - 0s - loss: 7480.5337
Epoch 18/200
 - 0s - loss: 6086.4731
Epoch 19/200
 - 0s - loss: 4695.7754
Epoch 20/200
 - 0s - loss: 3310.9316
Epoch 21/200
 - 0s - loss: 2053.6101
Epoch 22/200
 - 0s - loss: 1087.5088
Epoch 23/200
 - 0s - loss: 519.7864
Epoch 24/200
 - 0s - loss: 528.3585
Epoch 25/200
 - 0s - loss: 1056.0919
Epoch 26/200
 - 0s - loss: 1576.5482
Epoch 27/200
 - 0s - loss: 1736.17

In [0]:
x_input = array([[70, 75], [80, 85], [90, 95]])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[186.3042  207.68997]]
